In [1]:
import pandas as pd
import numpy as np

In [2]:
#pulling in job csv
jobs_ = "../data/Job Postings.csv"
jobs_df = pd.read_csv(jobs_)

In [3]:
#pulling in covid/merchant csv
covidmerchant_ = "../output_files/covidmerchant.csv"
covidmerchant_df = pd.read_csv(covidmerchant_)

In [4]:
#merging jobs/covid/merchants
covidmerchantjobs_df = pd.merge(covidmerchant_df,jobs_df, on=['cityid'], how='outer', suffixes=('_cvm','_jobs'))

In [5]:
#changing cityname to City Name to match cost of living
cv19bizjob_df = covidmerchantjobs_df.rename(columns={"cityname": "City Name"})

In [6]:
#pulling in living csv
living_ = "../output_files/cost_of_living.csv"
living_df = pd.read_csv(living_)

In [7]:
#adding 1 bedroom true cost (if needed)
living_df["1 Bedroom True Cost"]=living_df.iloc[:,[1,3]].sum(axis=1)

#adding 3 bedroom true cost (if needed)
living_df["3 Bedroom True Cost"]=living_df.iloc[:,[2,3]].sum(axis=1)

#adding Average true cost (if needed)
living_df["Avg True Cost"]=living_df.iloc[:,[4,5]].mean(axis=1).round(2)

In [8]:
#merging all info together - covid/merhcnat/jobs/living
#replacing Nan values with "Data Not Available"
allinfo_df = pd.merge(cv19bizjob_df,living_df, on=['City Name'], how='outer')
allinfo_df = allinfo_df.replace({np.NaN: "Data Not Available"})

In [9]:
#keeping cityid lat lon citypop merchantsall(change to%) newcaserate bgpost avetrucost
allinfo_df = allinfo_df.drop(['year',
                        'month',
                        'day',
                        'Cost of 1 Bedroom Appartment',
                        'Cost of 3 Bedroom Appartment',
                        'Utilities Per Month',
                        '1 Bedroom True Cost',
                        '3 Bedroom True Cost',
                        'Merchants Difference',
                        'Difference plus 1'], axis = 1)
                       

In [10]:
#pulling in covid/merchant csv
blsgeo_ = "../output_files/BLS_GEO_OCCUPATION_ALL_df_merge.csv"
blsgeo_df = pd.read_csv(blsgeo_,index_col=0)


In [11]:
#keeping geocitystate mwe occtypecode occtype skilllevel howpaid cityid
blsgeo_df = blsgeo_df.drop(['year',
                        'cityname',
                       'stateabbrev',
                        'statefips',
                        'lat',
                        'lon',
                        'city_pop2019',
                        'statename'], axis = 1)

In [12]:
#using a pivot table
ndf = blsgeo_df.pivot(index = "cityid", columns = 'MWE_Occupation Type', values = 'MWE')

In [13]:
#merging last df with pivot df
alldfs = pd.merge(allinfo_df,ndf, on=['cityid'], how='outer')

In [14]:
#replacing all values not accounted for as "Data Not Available"
alldfs = alldfs.replace({'Not Found': "Data Not Available", "-": "Data Not Available"})

In [15]:
#formatting the merchants_all column to percentage
alldfs['merchants_all'] = pd.Series(["{0:.2f}%".format(val * 100) for val in alldfs['merchants_all']], index = alldfs.index)

In [16]:
#import and clean hospital data
health_ = "../output_files/HealthData.csv"
health_df = pd.read_csv(health_)
#merge in hospital data
alldfs = pd.merge(health_df, alldfs, on=['cityid'], how='outer')


In [17]:
#droping more unneeded columns
alldfs = alldfs.drop(['Avg True Cost', 'Total Cost of 3 BHK ', 'Total Cost for 1 BHK '], axis = 1)
alldfs.columns

Index(['cityid', 'Health score', 'City Name', 'stateabbrev', 'lat', 'lon',
       'city_pop2019', 'merchants_all', 'new_case_rate', 'bg_posts',
       'Average Cost of Living ', 'Architecture and engineering occupations',
       'Arts, design, entertainment, sports, and media occupations',
       'Building and grounds cleaning and maintenance occupations',
       'Business and financial operations occupations',
       'Community and social service occupations',
       'Computer and mathematical occupations',
       'Construction and extraction occupations',
       'Educational instruction and library occupations',
       'Food preparation and serving related occupations',
       'Healthcare practitioners and technical occupations',
       'Healthcare support occupations',
       'Installation, maintenance, and repair occupations',
       'Legal occupations', 'Life, physical, and social science occupations',
       'Management occupations',
       'Office and administrative support occu

In [18]:
parks_ = "../output_files/parks_score.csv"
parks_df = pd.read_csv(parks_)
alldfs = pd.merge(parks_df, alldfs, on=['cityid'], how='outer')

In [19]:
# Rename a couple columns
alldfs.rename(columns={'Average Cost of Living ': 'Average Cost of Living', 'Health score': 'Health Score',\
                       'city_pop2019_y':'City pop2019'})
alldfs.columns

Index(['cityid', 'Park Score', 'Health score', 'City Name', 'stateabbrev',
       'lat', 'lon', 'city_pop2019', 'merchants_all', 'new_case_rate',
       'bg_posts', 'Average Cost of Living ',
       'Architecture and engineering occupations',
       'Arts, design, entertainment, sports, and media occupations',
       'Building and grounds cleaning and maintenance occupations',
       'Business and financial operations occupations',
       'Community and social service occupations',
       'Computer and mathematical occupations',
       'Construction and extraction occupations',
       'Educational instruction and library occupations',
       'Food preparation and serving related occupations',
       'Healthcare practitioners and technical occupations',
       'Healthcare support occupations',
       'Installation, maintenance, and repair occupations',
       'Legal occupations', 'Life, physical, and social science occupations',
       'Management occupations',
       'Office and admini

In [20]:
alldfs.set_index('cityid')


Park Score  Health score         City Name stateabbrev    lat     lon  \
cityid                                                                          
1         4.347368      4.233333       Los Angeles          CA  34.05 -118.24   
2         4.585000      3.784211     New York City          NY  40.71  -74.01   
3         4.676471      4.077778           Chicago          IL  41.88  -87.63   
4         4.529412      4.018182           Houston          TX  29.76  -95.37   
5         4.425000      3.983333           Phoenix          AZ  33.45 -112.07   
6         4.457895      3.953846         San Diego          CA  32.72 -117.16   
7         4.505882      3.375000            Dallas          TX  32.78  -96.80   
8         4.353333      3.266667         Las Vegas          NV  36.17 -115.14   
9         4.106667      3.468750           Seattle          WA  47.61 -122.33   
10        4.444444      4.100000        Fort Worth          TX  32.76  -97.33   
11        4.194737      3.586667       San Antonio          TX  29.42  -98.49   
12        3.942105      4.614286          San Jose          CA  37.34 -121.89   
13        4.400000      3.560000           Detroit          MI  42.33  -83.05   
14        4.460000      3.578571      Philadelphia          PA  39.95  -75.17   
15        4.511111      3.107692          Columbus          OH  39.96  -83.00   
16        4.221053      3.280000            Austin          TX  30.27  -97.74   
17        4.335294      4.000000         Charlotte          NC  35.23  -80.84   
18        4.440000      3.945455      Indianapolis          IN  39.77  -86.16   
19        4.188889      3.744444      Jacksonville          FL  30.33  -81.66   
20        4.255000      3.909091           Memphis          TN  35.15  -90.05   
21        4.435000      4.150000     San Francisco          CA  37.77 -122.42   
22        4.430000      3.400000           El Paso          TX  31.78 -106.44   
23        4.331579      3.705882         Baltimore          MD  39.29  -76.61   
24        4.247059      4.600000          Portland          OR  45.52 -122.68   
25        4.610526      4.233333            Boston          MA  42.36  -71.06   
26        4.557143      3.910000     Oklahoma City          OK  35.47  -97.52   
27        4.429412      3.463636        Louisville          KY  38.25  -85.76   
28        4.026667      3.284615            Denver          CO  39.74 -104.99   
29        4.325000      3.505882        Washington          DC  38.91  -77.04   
30        4.494118      4.200000         Nashville          TN  36.16  -86.78   
31        4.287500      3.984615         Milwaukee          WI  43.04  -87.91   
32        4.318182      3.820000       Albuquerque          NM  35.09 -106.61   
33        4.221053      4.333333            Tucson          AZ  32.22 -110.93   
34        3.985714      3.837500            Fresno          CA  36.75 -119.77   
35        4.455556      3.892308        Sacramento          CA  38.58 -121.49   
36        4.418750      4.178571           Atlanta          GA  33.75  -84.39   
37        4.378947      3.744444       Kansas City          MO  39.10  -94.58   
38        4.512500      4.411111             Miami          FL  25.76  -80.19   
39        4.594737      4.480000           Raleigh          NC  35.78  -78.64   
40        4.310000      4.085714             Omaha          NE  41.25  -96.00   
41        4.568421      3.254545           Oakland          CA  37.80 -122.27   
42        4.435000      3.100000       Minneapolis          MN  44.98  -93.27   
43        4.331579      3.933333             Tampa          FL  27.95  -82.46   
44        4.100000      4.344444       New Orleans          LA  29.95  -90.07   
45        4.400000      3.300000           Wichita          KS  37.69  -97.34   
46        4.194118      4.300000         Cleveland          OH  41.50  -81.69   
47        4.141667      4.200000       Bakersfield          CA  35.37 -119.02   
48        4.090000      4.061538       

In [21]:
# in order to do ranking we need NaN to be in place
alldfs = alldfs.replace({"Data Not Available": float('NaN')})


# now creating another data frame to hold rankings. 
column_list = ['Park Score', 'Health score', 'city_pop2019', 'merchants_all', 'bg_posts',
       'Average Cost of Living ', 'Architecture and engineering occupations',
       'Arts, design, entertainment, sports, and media occupations',
       'Building and grounds cleaning and maintenance occupations',
       'Business and financial operations occupations',
       'Community and social service occupations',
       'Computer and mathematical occupations',
       'Construction and extraction occupations',
       'Educational instruction and library occupations',
       'Food preparation and serving related occupations',
       'Healthcare practitioners and technical occupations',
       'Healthcare support occupations',
       'Installation, maintenance, and repair occupations',
       'Legal occupations', 'Life, physical, and social science occupations',
       'Management occupations',
       'Office and administrative support occupations',
       'Personal care and service occupations', 'Production occupations',
       'Protective service occupations', 'Sales and related occupations',
       'Transportation and material moving occupations']
for i in column_list:
    print(i)
    alldfs[i+"_RANK"] = alldfs[i].rank(ascending = False, na_option='bottom')
alldfs['new_case_rate'+"_RANK"] = alldfs['new_case_rate'].rank(ascending = True)

Park Score
Health score
city_pop2019
merchants_all
bg_posts
Average Cost of Living 
Architecture and engineering occupations
Arts, design, entertainment, sports, and media occupations
Building and grounds cleaning and maintenance occupations
Business and financial operations occupations
Community and social service occupations
Computer and mathematical occupations
Construction and extraction occupations
Educational instruction and library occupations
Food preparation and serving related occupations
Healthcare practitioners and technical occupations
Healthcare support occupations
Installation, maintenance, and repair occupations
Legal occupations
Life, physical, and social science occupations
Management occupations
Office and administrative support occupations
Personal care and service occupations
Production occupations
Protective service occupations
Sales and related occupations
Transportation and material moving occupations


In [22]:
# We also wanted to have an overall rank and overall score
rank_columns = [ 'Health score_RANK','Park Score_RANK',
       'city_pop2019_RANK', 'merchants_all_RANK', 'bg_posts_RANK',
       'Average Cost of Living _RANK',
       'new_case_rate_RANK']
alldfs['overall_SCORE'] = alldfs[[col for col in rank_columns]].sum(axis=1)
alldfs['overall_RANK'] = alldfs['overall_SCORE'].rank()


alldfs.columns

Index(['cityid', 'Park Score', 'Health score', 'City Name', 'stateabbrev',
       'lat', 'lon', 'city_pop2019', 'merchants_all', 'new_case_rate',
       'bg_posts', 'Average Cost of Living ',
       'Architecture and engineering occupations',
       'Arts, design, entertainment, sports, and media occupations',
       'Building and grounds cleaning and maintenance occupations',
       'Business and financial operations occupations',
       'Community and social service occupations',
       'Computer and mathematical occupations',
       'Construction and extraction occupations',
       'Educational instruction and library occupations',
       'Food preparation and serving related occupations',
       'Healthcare practitioners and technical occupations',
       'Healthcare support occupations',
       'Installation, maintenance, and repair occupations',
       'Legal occupations', 'Life, physical, and social science occupations',
       'Management occupations',
       'Office and admini

In [23]:
#resetting column order
fdf = alldfs[['cityid', 'City Name', 'stateabbrev', 'overall_RANK',
        'city_pop2019','city_pop2019_RANK', 'merchants_all',  'merchants_all_RANK',
        'new_case_rate', 'new_case_rate_RANK','bg_posts', 'bg_posts_RANK',
        'Average Cost of Living ', 'Average Cost of Living _RANK',
        'Architecture and engineering occupations', 'Architecture and engineering occupations_RANK',
        'Arts, design, entertainment, sports, and media occupations', 
        'Arts, design, entertainment, sports, and media occupations_RANK',
        'Building and grounds cleaning and maintenance occupations',
        'Building and grounds cleaning and maintenance occupations_RANK',
        'Business and financial operations occupations', 'Business and financial operations occupations_RANK',
        'Community and social service occupations', 'Community and social service occupations_RANK',
        'Computer and mathematical occupations', 'Computer and mathematical occupations_RANK',
        'Construction and extraction occupations', 'Construction and extraction occupations_RANK',
        'Educational instruction and library occupations', 'Educational instruction and library occupations_RANK',
        'Food preparation and serving related occupations', 'Food preparation and serving related occupations_RANK',
        'Healthcare practitioners and technical occupations', 'Healthcare practitioners and technical occupations_RANK',
        'Healthcare support occupations', 'Healthcare support occupations_RANK',
        'Installation, maintenance, and repair occupations', 'Installation, maintenance, and repair occupations_RANK',
        'Legal occupations', 'Legal occupations_RANK',
        'Life, physical, and social science occupations', 'Life, physical, and social science occupations_RANK',
        'Management occupations', 'Management occupations_RANK',
        'Office and administrative support occupations', 'Office and administrative support occupations_RANK',
        'Personal care and service occupations', 'Personal care and service occupations_RANK',
        'Production occupations', 'Production occupations_RANK',
        'Protective service occupations', 'Protective service occupations_RANK',
        'Sales and related occupations', 'Sales and related occupations_RANK',
        'Transportation and material moving occupations', 'Transportation and material moving occupations_RANK',
        'Park Score', 'Park Score_RANK',
        'Health score', 'Health score_RANK', 'overall_SCORE', 'lat', 'lon']]

In [24]:
fdf.to_csv('../output_files/MegaDf.csv', index = False)